# Hand Joint Angle Estimation

Estimating the angle between hand joints is an important task in various fields such as computer vision, robotics, augmented reality, and biomedical engineering. 
The goal is to determine the angles formed by the bones of the hand at each joint, which can be useful for applications like gesture recognition, hand tracking, sign language interpretation, and prosthetics control.


## 1. Importing all the necessary libraries

In [ ]:
if 'google.colab' in str(get_ipython()):
    print("Downloading Code to Colab Environment")
    !wget https://www.dropbox.com/sh/gz8x999ela84qr2/AABHRkHY_QALmo6JHSpOruA8a?dl=1 -O module-code.zip -q --show-progress
    !unzip -qq module-code.zip
    !pip install --upgrade opencv-contrib-python
    !pip install mediapipe
    %cd Applications/
else:
    pass


In [ ]:
import cv2
import time
import math
import mediapipe as mp
import numpy as np
import os
import json
ffmpeg_path = '/Users/krishnakanth/Downloads/DFKI HiWi'#Set this path to the working directory of the notebook file
os.environ['IMAGEIO_FFMPEG_EXE'] = ffmpeg_path
from moviepy.editor import VideoFileClip
from IPython.display import display

Setting os.environ['IMAGEIO_FFMPEG_EXE'] to ffmpeg_path is important because it specifies the path to the ffmpeg executable for the imageio library to use, ensuring proper functionality for video processing tasks within Python. 

## 2. Initializations

### <font style="color:rgb(50,120,230)">Create video capture object<font/>
- The video of a moving hand has to be saved in a specific directory
- Make sure that the path for the directory containing the saved video of the hand is addressed correctly in 
    the following step

In [ ]:
file_name  = '/Users/krishnakanth/Downloads/DFKI HiWi/Hand pose estimation.mp4'
video_cap = cv2.VideoCapture(file_name)
if not video_cap.isOpened():
    print('Unable to open: ' + file_name)

### <font style="color:rgb(50,120,230)">Create video writer object<font/>
- This oject is used in the later stages to save the annotated video
- The annotated video is saved in the same directory as the input video

In [ ]:
fps = int(video_cap.get(cv2.CAP_PROP_FPS))
frame_w = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (frame_w, frame_h)

video_out_file = file_name[:-4] + '_out_landmarks.mp4'
video_output = cv2.VideoWriter(video_out_file, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)

# 3. Extract Landmark Coordinates

### <font style="color:rgb(50,120,230)">Process image frames and extract landmark coordinates<font/>
<br>
<center>
<img src = "https://ai.google.dev/static/mediapipe/images/solutions/hand-landmarks.png"
</center>
<br>
    [Source: Google Al for Developers]

- In this step the input video is annotated with joints and lines connecting the joints
- The annotated video is exported 

In [ ]:
# BGR Colors
color_marks  = (0, 0, 255)
color_join   = (0, 255, 0)


mp_hands = mp.solutions.hands

with mp_hands.Hands(min_detection_confidence=0.1, min_tracking_confidence=0.1) as hands:
    while True:
        has_frame, frame = video_cap.read()
        if not has_frame:
            break
            
        # Convert the BGR image to RGB.
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image frame.
        keypoints = hands.process(frame)
        landmarks = keypoints.multi_hand_landmarks
        
        # Convert the image back to BGR.
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        if landmarks:
            for hand_landmarks in landmarks:
                # Wrist
                wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame_w)
                wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame_h)
                wrist_p = np.array([wrist_x, wrist_y])

                # Thumb
                thumb_cmc_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC].x * frame_w)
                thumb_cmc_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC].y * frame_h)
                thumb_cmc_p = np.array([thumb_cmc_x, thumb_cmc_y])

                thumb_mcp_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].x * frame_w)
                thumb_mcp_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].y * frame_h)
                thumb_mcp_p = np.array([thumb_mcp_x, thumb_mcp_y])

                thumb_ip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].x * frame_w)
                thumb_ip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].y * frame_h)
                thumb_ip_p = np.array([thumb_ip_x, thumb_ip_y])

                thumb_tip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * frame_w)
                thumb_tip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * frame_h)
                thumb_tip_p = np.array([thumb_tip_x, thumb_tip_y])

                # Index Finger
                index_mcp_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x * frame_w)
                index_mcp_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y * frame_h)
                index_mcp_p = np.array([index_mcp_x, index_mcp_y])

                index_pip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x * frame_w)
                index_pip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y * frame_h)
                index_pip_p = np.array([index_pip_x, index_pip_y])

                index_dip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].x * frame_w)
                index_dip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y * frame_h)
                index_dip_p = np.array([index_dip_x, index_dip_y])

                index_tip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * frame_w)
                index_tip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * frame_h)
                index_tip_p = np.array([index_tip_x, index_tip_y])

                # Middle Finger
                middle_mcp_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x * frame_w)
                middle_mcp_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y * frame_h)
                middle_mcp_p = np.array([middle_mcp_x, middle_mcp_y])

                middle_pip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x * frame_w)
                middle_pip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y * frame_h)
                middle_pip_p = np.array([middle_pip_x, middle_pip_y])

                middle_dip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].x * frame_w)
                middle_dip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y * frame_h)
                middle_dip_p = np.array([middle_dip_x, middle_dip_y])

                middle_tip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * frame_w)
                middle_tip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * frame_h)
                middle_tip_p = np.array([middle_tip_x, middle_tip_y])

                # Ring Finger
                ring_mcp_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].x * frame_w)
                ring_mcp_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y * frame_h)
                ring_mcp_p = np.array([ring_mcp_x, ring_mcp_y])

                ring_pip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].x * frame_w)
                ring_pip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y * frame_h)
                ring_pip_p = np.array([ring_pip_x, ring_pip_y])

                ring_dip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].x * frame_w)
                ring_dip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y * frame_h)
                ring_dip_p = np.array([ring_dip_x, ring_dip_y])

                ring_tip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x * frame_w)
                ring_tip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * frame_h)
                ring_tip_p = np.array([ring_tip_x, ring_tip_y])

                # Pinky Finger
                pinky_mcp_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].x * frame_w)
                pinky_mcp_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].y * frame_h)
                pinky_mcp_p = np.array([pinky_mcp_x, pinky_mcp_y])

                pinky_pip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].x * frame_w)
                pinky_pip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y * frame_h)
                pinky_pip_p = np.array([pinky_pip_x, pinky_pip_y])

                pinky_dip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].x * frame_w)
                pinky_dip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y * frame_h)
                pinky_dip_p = np.array([pinky_dip_x, pinky_dip_y])

                pinky_tip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x * frame_w)
                pinky_tip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y * frame_h)
                pinky_tip_p = np.array([pinky_tip_x, pinky_tip_y])
                # Join landmarks.
                # Drawing lines for the thumb
                cv2.line(frame, (wrist_p[0], wrist_p[1] ), (thumb_cmc_p[0], thumb_cmc_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (thumb_cmc_p[0], thumb_cmc_p[1] ), (thumb_mcp_p[0], thumb_mcp_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (thumb_mcp_p[0], thumb_mcp_p[1]), (thumb_ip_p[0], thumb_ip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (thumb_ip_p[0], thumb_ip_p[1]), (thumb_tip_p[0], thumb_tip_p[1]), color_join, 2, cv2.LINE_AA)

                # Drawing lines for the index finger
                cv2.line(frame, (wrist_p[0], wrist_p[1]), (index_mcp_p[0], index_mcp_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (index_mcp_p[0], index_mcp_p[1]), (index_pip_p[0], index_pip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (index_pip_p[0], index_pip_p[1]), (index_dip_p[0], index_dip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (index_dip_p[0], index_dip_p[1]), (index_tip_p[0], index_tip_p[1]), color_join, 2, cv2.LINE_AA)

                # Drawing lines for the middle finger
                
                cv2.line(frame, (middle_mcp_p[0], middle_mcp_p[1]), (middle_pip_p[0], middle_pip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (middle_pip_p[0], middle_pip_p[1]), (middle_dip_p[0], middle_dip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (middle_dip_p[0], middle_dip_p[1]), (middle_tip_p[0], middle_tip_p[1]), color_join, 2, cv2.LINE_AA)

                # Drawing lines for the ring finger
                
                cv2.line(frame, (ring_mcp_p[0], ring_mcp_p[1]), (ring_pip_p[0], ring_pip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (ring_pip_p[0], ring_pip_p[1]), (ring_dip_p[0], ring_dip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (ring_dip_p[0], ring_dip_p[1]), (ring_tip_p[0], ring_tip_p[1]), color_join, 2, cv2.LINE_AA)

                # Drawing lines for the pinky finger
                cv2.line(frame, (wrist_p[0], wrist_p[1]), (pinky_mcp_p[0], pinky_mcp_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (pinky_mcp_p[0], pinky_mcp_p[1]), (pinky_pip_p[0], pinky_pip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (pinky_pip_p[0], pinky_pip_p[1]), (pinky_dip_p[0], pinky_dip_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (pinky_dip_p[0], pinky_dip_p[1]), (pinky_tip_p[0], pinky_tip_p[1]), color_join, 2, cv2.LINE_AA)

                # Drawing lines between MCP landmarks of the fingers
                cv2.line(frame, (index_mcp_p[0], index_mcp_p[1]), (middle_mcp_p[0], middle_mcp_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (middle_mcp_p[0], middle_mcp_p[1]), (ring_mcp_p[0], ring_mcp_p[1]), color_join, 2, cv2.LINE_AA)
                cv2.line(frame, (ring_mcp_p[0], ring_mcp_p[1]), (pinky_mcp_p[0], pinky_mcp_p[1]), color_join, 2, cv2.LINE_AA)
                # Draw landmarks
                # Draw circle at wrist
                cv2.circle(frame, (wrist_p[0], wrist_p[1]), 3, color_marks, -1)

                # Draw circles at thumb landmarks
                cv2.circle(frame, (thumb_cmc_p[0], thumb_cmc_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (thumb_mcp_p[0], thumb_mcp_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (thumb_ip_p[0], thumb_ip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (thumb_tip_p[0], thumb_tip_p[1]), 3, color_marks, -1)

                # Draw circles at index finger landmarks
                cv2.circle(frame, (index_mcp_p[0], index_mcp_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (index_pip_p[0], index_pip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (index_dip_p[0], index_dip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (index_tip_p[0], index_tip_p[1]), 3, color_marks, -1)

                # Draw circles at middle finger landmarks
                cv2.circle(frame, (middle_mcp_p[0], middle_mcp_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (middle_pip_p[0], middle_pip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (middle_dip_p[0], middle_dip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (middle_tip_p[0], middle_tip_p[1]), 3, color_marks, -1)

                # Draw circles at ring finger landmarks
                cv2.circle(frame, (ring_mcp_p[0], ring_mcp_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (ring_pip_p[0], ring_pip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (ring_dip_p[0], ring_dip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (ring_tip_p[0], ring_tip_p[1]), 3, color_marks, -1)

                # Draw circles at pinky finger landmarks
                cv2.circle(frame, (pinky_mcp_p[0], pinky_mcp_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (pinky_pip_p[0], pinky_pip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (pinky_dip_p[0], pinky_dip_p[1]), 3, color_marks, -1)
                cv2.circle(frame, (pinky_tip_p[0], pinky_tip_p[1]), 3, color_marks, -1)
                video_output.write(frame)
            
video_cap.release()
video_output.release()


# 4.  Defining Vectors Between Landmark Points
<br>
<center>
<img src = "https://ai.google.dev/static/mediapipe/images/solutions/examples/hand_landmark.png"
</center>
<br>
Vectors joining the landmark points of a hand [Source: Google Al for Developers]

# 5. Function to Compute the Angle Between Two Vectors

The dot product between two vectors is defined as follows:

\begin{align}
{a\bullet b} = {|a||b|}\bullet \cos(\theta)
\end{align}

We can therefore compute the angle between the two vectors re-arranging the above equation.


\begin{align}
\theta = \arccos (\frac{a\bullet b}{|a||b|})
\end{align}

In [ ]:
def compute_angle(v1, v2):

    # Unit vector.
    v1u = v1 / np.linalg.norm(v1)
    # Unit vector.
    v2u = v2 / np.linalg.norm(v2)
    # Compute the angle between the two unit vectors.
    angle_deg = np.arccos(np.dot(v1u, v2u)) * 180 / math.pi

    return angle_deg

# 6. Initializations

### <font style="color:rgb(50,120,230)">Create Video Capture Object<font/>
- The video of a moving hand has to be saved in a specific directory
- Make sure that the path for the directory containing the saved video of the hand is addressed correctly in 
    the following step

In [ ]:
file_name  = '/Users/krishnakanth/Downloads/DFKI HiWi/Hand pose estimation.mp4'
video_cap = cv2.VideoCapture(file_name)
if not video_cap.isOpened():
    print('Unable to open: ' + file_name)

# 7. Convenience Function to Extract Landmark Coordinates

In [ ]:
def get_landmark_point(landmarks, landmark_point, w, h):
    x = int(landmarks.landmark[landmark_point].x * w)
    y = int(landmarks.landmark[landmark_point].y * h)
    point = np.array([x, y])
    return point

# 8. Joint angles calculation and export

### <font style="color:rgb(50,120,230)">Process image frames and extract landmark coordinates<font/>
<br>
<center>
<img src = "https://ai.google.dev/static/mediapipe/images/solutions/hand-landmarks.png"
</center>
<br>
[Source: Google Al for Developers]

- The angles at the joints are calculated and exported as a .json file into the same directory as the input video

In [ ]:
first_frame = True

    
mp_hand = mp.solutions.hands

angles_data = []
    
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        
    while True:

        has_frame, frame = video_cap.read()
        if not has_frame:
            break

        # Convert the BGR image to RGB.
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image frame.
        keypoints = hands.process(frame)
        landmarks = keypoints.multi_hand_landmarks
        
        # Convert the image back to BGR.
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        


        if landmarks is not None:
            for hand_landmarks in landmarks:
                # Acquire the landmark coordinates for the hand.
                wrist_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.WRIST, frame_w, frame_h)
                thumb_cmc_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.THUMB_CMC, frame_w, frame_h)
                thumb_mcp_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.THUMB_MCP, frame_w, frame_h)
                thumb_ip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.THUMB_IP, frame_w, frame_h)
                thumb_tip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.THUMB_TIP, frame_w, frame_h)
                index_finger_mcp_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.INDEX_FINGER_MCP, frame_w, frame_h)
                index_finger_pip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.INDEX_FINGER_PIP, frame_w, frame_h)
                index_finger_dip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.INDEX_FINGER_DIP, frame_w, frame_h)
                index_finger_tip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.INDEX_FINGER_TIP, frame_w, frame_h)
                middle_finger_mcp_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.MIDDLE_FINGER_MCP, frame_w, frame_h)
                middle_finger_pip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.MIDDLE_FINGER_PIP, frame_w, frame_h)
                middle_finger_dip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.MIDDLE_FINGER_DIP, frame_w, frame_h)
                middle_finger_tip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, frame_w, frame_h)
                ring_finger_mcp_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.RING_FINGER_MCP, frame_w, frame_h)
                ring_finger_pip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.RING_FINGER_PIP, frame_w, frame_h)
                ring_finger_dip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.RING_FINGER_DIP, frame_w, frame_h)
                ring_finger_tip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.RING_FINGER_TIP, frame_w, frame_h)
                pinky_mcp_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.PINKY_MCP, frame_w, frame_h)
                pinky_pip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.PINKY_PIP, frame_w, frame_h)
                pinky_dip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.PINKY_DIP, frame_w, frame_h)
                pinky_tip_p = get_landmark_point(hand_landmarks, mp_hands.HandLandmark.PINKY_TIP, frame_w, frame_h)


                # Compute the vectors.
                v_wrist_thumb_cmc = np.subtract(wrist_p, thumb_cmc_p)
                v_thumb_cmc_thumb_mcp = np.subtract(thumb_cmc_p, thumb_mcp_p)
                v_thumb_mcp_thumb_ip = np.subtract(thumb_mcp_p, thumb_ip_p)
                v_thumb_ip_thumb_tip = np.subtract(thumb_ip_p, thumb_tip_p)
                v_wrist_index_mcp = np.subtract(wrist_p, index_finger_mcp_p)
                v_index_mcp_index_pip = np.subtract(index_finger_mcp_p, index_finger_pip_p)
                v_index_pip_index_dip = np.subtract(index_finger_pip_p, index_finger_dip_p)
                v_index_dip_index_tip = np.subtract(index_finger_dip_p, index_finger_tip_p)
                v_wrist_middle_mcp = np.subtract(wrist_p, middle_finger_mcp_p)
                v_middle_mcp_middle_pip = np.subtract(middle_finger_mcp_p, middle_finger_pip_p)
                v_middle_pip_middle_dip = np.subtract(middle_finger_pip_p, middle_finger_dip_p)
                v_middle_dip_middle_tip = np.subtract(middle_finger_dip_p, middle_finger_tip_p)
                v_wrist_ring_mcp = np.subtract(wrist_p, ring_finger_mcp_p)
                v_ring_mcp_ring_pip = np.subtract(ring_finger_mcp_p, ring_finger_pip_p)
                v_ring_pip_ring_dip = np.subtract(ring_finger_pip_p, ring_finger_dip_p)
                v_ring_dip_ring_tip = np.subtract(ring_finger_dip_p, ring_finger_tip_p)
                v_wrist_pinky_mcp = np.subtract(wrist_p, pinky_mcp_p)
                v_pinky_mcp_pinky_pip = np.subtract(pinky_mcp_p, pinky_pip_p)
                v_pinky_pip_pinky_dip = np.subtract(pinky_pip_p, pinky_dip_p)
                v_pinky_dip_pinky_tip = np.subtract(pinky_dip_p, pinky_tip_p)
                
                # Compute the angles (THE ANGLES ARE CALCULATED ONLY FOR THE JOINTS WITH ONLY TWO VECTORS)
                angle_joint_1 = compute_angle(v_wrist_thumb_cmc, v_thumb_cmc_thumb_mcp)
                angle_joint_2 = compute_angle(v_thumb_cmc_thumb_mcp, v_thumb_mcp_thumb_ip)
                angle_joint_3 = compute_angle(v_thumb_mcp_thumb_ip, v_thumb_ip_thumb_tip)
                angle_joint_5 = compute_angle(v_wrist_index_mcp, v_index_mcp_index_pip)
                angle_joint_6 = compute_angle(v_index_mcp_index_pip, v_index_pip_index_dip)
                angle_joint_7 = compute_angle(v_index_pip_index_dip, v_index_dip_index_tip)
                angle_joint_10 = compute_angle(v_middle_mcp_middle_pip, v_middle_pip_middle_dip)
                angle_joint_11 = compute_angle(v_middle_pip_middle_dip, v_middle_dip_middle_tip)
                angle_joint_14 = compute_angle(v_ring_mcp_ring_pip, v_ring_pip_ring_dip)
                angle_joint_15 = compute_angle(v_ring_pip_ring_dip, v_ring_dip_ring_tip)
                angle_joint_18 = compute_angle(v_pinky_mcp_pinky_pip, v_pinky_pip_pinky_dip)
                angle_joint_19 = compute_angle(v_pinky_pip_pinky_dip, v_pinky_dip_pinky_tip)
                # Create dictionary for angles data
                angles_entry = {
                    "angle_joint_1": angle_joint_1,
                    "angle_joint_2": angle_joint_2,
                    "angle_joint_3": angle_joint_3,
                    "angle_joint_5": angle_joint_5,
                    "angle_joint_6": angle_joint_6,
                    "angle_joint_7": angle_joint_7,
                    "angle_joint_10": angle_joint_10,
                    "angle_joint_11": angle_joint_11,
                    "angle_joint_14": angle_joint_14,
                    "angle_joint_15": angle_joint_15,
                    "angle_joint_18": angle_joint_18,
                    "angle_joint_19": angle_joint_19
                }
                # Append entry to angles_data list
                angles_data.append(angles_entry)

# Write angles_data to JSON file
output_file = 'angles_data.json'
with open(output_file, 'w') as json_file:
    json.dump(angles_data, json_file, indent=4)


print(f"Angles data has been successfully exported to {output_file}")